# Spark Preparation
We check if we are in Google Colab.  If this is the case, install all necessary packages.

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.3.2 with hadoop 3.3, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab.
Learn more from [A Must-Read Guide on How to Work with PySpark on Google Colab for Data Scientists!](https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/)

In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [7]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q https://dlcdn.apache.org/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz
    !tar xf spark-3.5.5-bin-hadoop3.tgz
    !mv spark-3.5.5-bin-hadoop3 spark
    !pip install -q findspark
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark"

# Start a Local Cluster

In [8]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Movie Analysis") \
    .getOrCreate()

# Spark Assignment

Based on the movie review dataset in 'netflix-rotten-tomatoes-metacritic-imdb.csv', answer the below questions.

**Note:** do not clean or remove missing data

In [14]:
df = spark.read.option("header", "true").csv("netflix-rotten-tomatoes-metacritic-imdb.csv")

## What is the maximum and average of the overall hidden gem score?

In [15]:
from pyspark.sql.functions import max, avg, col

hidden_gem_stats = df.select(
    max(col("Hidden Gem Score")).alias("max_hidden_gem"),
    avg(col("Hidden Gem Score")).alias("avg_hidden_gem")
)

print("Maximum and Average Hidden Gem Score:")
hidden_gem_stats.show()

Maximum and Average Hidden Gem Score:
+--------------+-----------------+
|max_hidden_gem|   avg_hidden_gem|
+--------------+-----------------+
|           9.8|5.937551386501232|
+--------------+-----------------+



## How many movies that are available in Korea?

In [16]:
from pyspark.sql.functions import col

korea_count = df.filter(col("Country Availability").contains("Korea")).count()

print(f"Number of movies available in Korea: {korea_count}")

Number of movies available in Korea: 4845


## Which director has the highest average hidden gem score?

In [17]:
from pyspark.sql.functions import mean

director_scores = df.groupBy("Director") \
    .agg(mean("Hidden Gem Score").alias("avg_hidden_gem_score")) \
    .orderBy(col("avg_hidden_gem_score").desc()) \
    .limit(1)

print("Director with highest average hidden gem score:")
director_scores.show()

Director with highest average hidden gem score:
+-----------+--------------------+
|   Director|avg_hidden_gem_score|
+-----------+--------------------+
|Dorin Marcu|                 9.8|
+-----------+--------------------+



## How many genres are there in the dataset?

In [18]:
from pyspark.sql.functions import split, explode

genre_count = df.select(
    explode(split(col("Genre"), ", ")).alias("genre")
).distinct().count()

print(f"Number of unique genres: {genre_count}")

Number of unique genres: 28


In [19]:
spark.stop()